In [6]:
import json
import numpy as np

In [7]:
_instr_json = json.load(open('./instructions-train.json'))
_scene_data = json.load(open('./scenes-train.json'))

In [8]:
instr_json = _instr_json["instructions"]
scene_data = _scene_data['scenes']

# T5 data

In [4]:
# scene_data[0][0]['objects']


In [ ]:
# instr_json[0].keys()

In [4]:
# data = []
# for idx, instr in enumerate(instr_json):
#     sent = instr_json[idx]['instruction']
#     gprog = instr_json[idx]['grounded_program']
#     parse = ""
#     for prog in gprog:
#         parse += f"ASSIGN_HEAD(Unique(Filter({scene_data[idx][0]['objects'][prog[2]]['color']}, Filter({scene_data[idx][0]['objects'][prog[2]]['type']}, World)))) ; MOVE_HEAD({prog[0]}) ; KEEP(Filter({scene_data[idx][0]['objects'][prog[1]]['color']}, Filter({scene_data[idx][0]['objects'][prog[1]]['type']}, World))) ;"
#     data.append([sent, parse])

In [9]:
data = []
for idx, instr in enumerate(instr_json):
    sent = instr_json[idx]['instruction']
    gprog = instr_json[idx]['grounded_program']
    parse = ""
    # if prog[4] checks whether a scene was generated for this action. 
    # We only create parsed instructions for actions that are demonstrated through scenes. 
    for prog in gprog:
        if(prog[4]):
            move_head_part = f"MOVE_HEAD({prog[0]}) ; "
            parse += f"ASSIGN_HEAD(Unique(Filter({scene_data[idx][0]['objects'][prog[2]]['color']}, Filter({scene_data[idx][0]['objects'][prog[2]]['type']}, World)))) ; {prog[3] * move_head_part}KEEP(Filter({scene_data[idx][0]['objects'][prog[1]]['color']}, Filter({scene_data[idx][0]['objects'][prog[1]]['type']}, World))) ;"

    data.append([sent, parse])

In [10]:
with open("./data_t5-train.txt", 'w') as f:
    for d in data:
        f.write(f"{d[0]}\t{d[1]} \n")

# DataRemove

In [ ]:
import shutil
for dir in ['train', 'val', 'test']:
    instr_json = json.load(open(f'./instructions-{dir}.json'))
    instr_json = instr_json["instructions"]
    for entry in instr_json:
        if entry['template_json_filename'] == "SingleStep.json":
            !rm -r ./{dir}/{entry['ex_dir']}
            